#db-ally tutorial

In this tutorial, we are going to build a robust, explainable, and prompt-efficient chat agent, which will answer questions regarding the HR-recruitment database. We will use [**db-ally**](https://db-ally.deepsense.ai/) - `Efficient, consistent and secure library for querying structured data with natural language` and [SQLite](https://www.sqlite.org/) database served with the help of [sqlalchemy](https://www.sqlalchemy.org/).

Throughout the lesson, we will be working with the following HR database.


<img src="https://drive.google.com/uc?export=view&id=1A5yPt-pIyXGV94c6cMIkMf8AhiP6Nnq6" alt="Database schema" width="700" height="auto">


## Setup

Install [**db-ally**](https://db-ally.deepsense.ai/) with the openai extension.

In [ ]:
!pip install dbally[litellm] a-world-of-countries

Download the [`hr database file`](https://drive.google.com/file/d/1A5yPt-pIyXGV94c6cMIkMf8AhiP6Nnq6/view?usp=drive_link)

In [ ]:
!wget -O recruitment.db 'https://drive.google.com/uc?export=download&id=1zo3j8x7qH8opTKyQ9qFgRpS3yqU6uTRs'

Next, we need to:
1. Create the database (`ENGINE`). Thanks, [SQLite In-memory database](https://www.sqlite.org/inmemorydb.html) we can load the file directly.
2. Define ORM Model (`RECRUITMENT_MODEL`) with [sqlalchemy automap](https://docs.sqlalchemy.org/en/20/orm/extensions/automap.html)
3. Check that everything went well with [selecting](https://docs.sqlalchemy.org/en/20/tutorial/data_select.html) and printing 5 rows of every table

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import select

ENGINE = create_engine("sqlite:///recruitment.db")

RECRUITMENT_MODEL = automap_base()
RECRUITMENT_MODEL.prepare(autoload_with=ENGINE, reflect=True)

# Verify that all entires were created
stmts = [
    select(RECRUITMENT_MODEL.classes.candidate),
    select(RECRUITMENT_MODEL.classes.application),
    select(RECRUITMENT_MODEL.classes.offer),
]

with ENGINE.connect() as conn:
    for stmt in stmts:
        for row in conn.execute(stmt.limit(5)):
            print(row)

## Create the first view

Our database is ready! The next task is to create the first version of our agent and have a conversation 🗨

Let's:
1. Create a [View](https://db-ally.deepsense.ai/concepts/views/) in which we define functions accessible to the [Intermediate Query Language](https://db-ally.deepsense.ai/concepts/iql/) (IQL) engine.
2. Write `get_select` function that provides a foundation of the [`SELECT`](https://www.w3schools.com/sql/sql_select.asp) statement generated by the [IQL engine](https://db-ally.deepsense.ai/concepts/iql/)
3. Create our first filter `at_least_experience` using [ORM querying possibilities](https://docs.sqlalchemy.org/en/20/orm/queryguide/)

In [ ]:
from dbally import SqlAlchemyBaseView, decorators
import sqlalchemy


class CandidateView(SqlAlchemyBaseView):
    """
    View meant for answering questions just about the candidates.
    """

    def get_select(self) -> sqlalchemy.Select:
        return select(RECRUITMENT_MODEL.classes.candidate)

    @decorators.view_filter()
    def at_least_experience(self, years: int) -> sqlalchemy.ColumnElement:
        return RECRUITMENT_MODEL.classes.candidate.years_of_experience >= years

Now our agent can use this view to query the database. To accomplish this we need to create `ask_dbally` function:
1. Set up dbally to [use particular LLM](https://db-ally.deepsense.ai/how-to/use_custom_llm/) and provide the API_KEY
2. Set [CLIEventHandler](https://db-ally.deepsense.ai/how-to/create_custom_event_handler/) - This will print all intermediate conversation steps.
3. Create a [Collection](https://db-ally.deepsense.ai/concepts/collections/) that is a queryable container of different [Views](https://db-ally.deepsense.ai/concepts/views/).
4. Mark the fynction as `async`. It is necessary since Collection is [asynchronous](https://en.wikipedia.org/wiki/Asynchrony_(computer_programming)).
5. Execute `run_dbally`, providing it the question.

In [ ]:
from dbally.audit.event_handlers.cli_event_handler import CLIEventHandler
from dbally.llms.litellm import LiteLLM
import dbally

API_KEY = "API-KEY-GOES-HERE"


async def ask_dbally(question: str):
    llm = LiteLLM(api_key=API_KEY)
    recruitment_db = dbally.create_collection("recruitment", llm, event_handlers=[CLIEventHandler()])
    recruitment_db.add(CandidateView, lambda: CandidateView(ENGINE))

    await recruitment_db.ask(question)


await ask_dbally("Which candidates have more than 5 years of experience?")

You should see a conversation between you and the agent. Notice that:
1. LLM is given all filters we've created and instructions on how to use them (see [prompt description](docs) for more details)
```
at_least_experience(years: int)
```
2. LLM produced an [IQL](docs) query.
```
RESPONSE: at_least_experience(5)
```

3. Based on the output of 2. the [IQL engine](docs) generated a proper SQL that is later executed.

```sql
SELECT candidate.id, candidate.name, candidate.country, candidate.years_of_experience, candidate.position,         
candidate.university, candidate.skills, candidate.tags                                                             
FROM candidate                                                                                                     
WHERE candidate.years_of_experience >= 5   
```


### Ask an unsupported question

Pay attention to the part of the prompt instructing LLM to return `UnexpectedQueryError` if there are no such filters to answer the query. Let's test it.

In [ ]:
await ask_dbally("Who studied at Stanford?")

**Remark**: We should see `UnexpectedQueryError`. However, even though we state very strongly in the prompt that LLM should not fabricate any filter, it is non-deterministic, so hallucinations are still possible.

## Add more filters

Let's add more capabilities to our agent by defining the following filters:

1. `has_seniority` - Encodes domain knowledge specific to a given company.
2. `is_from_continent` - Uses an external API to inject knowledge about countries within continents.

In [ ]:
from typing import Literal

import awoc


class CandidateView(SqlAlchemyBaseView):
    """
    View meant for answering questions just about the candidates.
    """

    def get_select(self) -> sqlalchemy.Select:
        return select(RECRUITMENT_MODEL.classes.candidate)

    @decorators.view_filter()
    def at_least_experience(self, years: int) -> sqlalchemy.ColumnElement:
        return RECRUITMENT_MODEL.classes.candidate.years_of_experience >= years

    @decorators.view_filter()
    def has_seniority(self, seniority: Literal["junior", "senior", "mid"]) -> sqlalchemy.ColumnElement:
        """Allows filtering candidates based on their seniority"""
        if seniority == "junior":
            return RECRUITMENT_MODEL.classes.candidate.years_of_experience < 2
        if seniority == "mid":
            return RECRUITMENT_MODEL.classes.candidate.years_of_experience < 5

        return RECRUITMENT_MODEL.classes.candidate.years_of_experience >= 5

    @decorators.view_filter()
    def is_from_continent(
        self, continent: Literal["Europe", "Asia", "Africa", "North America", "South America", "Australia"]
    ) -> sqlalchemy.ColumnElement:
        my_world = awoc.AWOC()
        wanted_countries = my_world.get_countries_list_of(continent)
        return RECRUITMENT_MODEL.classes.candidate.country.in_(wanted_countries)

Ask  a question about senior candidates

In [ ]:
await ask_dbally("Are there any senior candidates?")

Once more, Let's take a quick look at the list of filters provided to the agent

```
at_least_experience(years: int)
has_seniority(seniority: Literal['junior', 'senior', 'mid']) Allows filtering candidates based on their seniority
is_from_continent(self, continent: Literal["Europe", "Asia", "Africa", "North America", "South America", "Australia"])
```

There are three things to be noticed:
1. Agent has access to more filters
2. All type hints are passed to the prompt automatically
3. Docstring of `has_seniority` function is available inside the prompt to allay any doubts.

Proceed and ask about developers based in Europe.

In [ ]:
await ask_dbally("Which developers are based in Europe?")

Notice the generated query

```sql
SELECT candidate.id, candidate.name, candidate.country, candidate.years_of_experience, candidate.position,         
candidate.university, candidate.skills, candidate.tags                                                             
FROM candidate                                                                                                     
WHERE candidate.country IN ('Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',       
'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Faroe Islands', 'Finland', 'France',     
'Germany', 'Gibraltar', 'Greece', 'Guernsey', 'Hungary', 'Iceland', 'Ireland', 'Isle of Man', 'Italy', 'Jersey',   
'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macedonia', 'Malta', 'Moldova', 'Monaco',         
'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia',          
'Slovakia', 'Slovenia', 'Spain', 'Svalbard and Jan Mayen', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom',   
'Vatican')
```

We didn't have to inject this knowledge into the prompt - we saved many tokens. Additionally, there is no chance of hallucinations (skipping or creating new countries) since everything is hardcoded inside the API.

## Combine conditions

So far, all [IQL](https://db-ally.deepsense.ai/concepts/iql/) expressions have been built from one filter. Normally, we are often interested in combinations of conditions. Let's examine the possibilities that we have

### Let the model do it

In [ ]:
await ask_dbally("Are there any senior candidates that are based in either of Americas?")

Next, look closely at the generated IQL

```sql
RESPONSE: has_seniority("senior") AND (is_from_continent("North America") OR is_from_continent("South America"))
```

[How does LLM now how to connect these filters?](https://db-ally.deepsense.ai/concepts/iql/)

### Code it by yourself

It may be tedious for the end user to write such convoluted queries manually. Imagine asking `Give me a candidate with 5 years of experience, from Europe, knowing Python, C++, and Machine Learning ... who is also a good team player`

To facilitate more direct questions, we utilize [AND SqlAlchemy functionality](https://docs.sqlalchemy.org/en/20/core/sqlelement.html) to create our next filter

In [ ]:
from sqlalchemy import and_


class CandidateView(SqlAlchemyBaseView):
    """
    View meant for answering questions just about the candidates.
    """

    def get_select(self) -> sqlalchemy.Select:
        return select(RECRUITMENT_MODEL.classes.candidate)

    @decorators.view_filter()
    def at_least_experience(self, years: int) -> sqlalchemy.ColumnElement:
        return RECRUITMENT_MODEL.classes.candidate.years_of_experience >= years

    @decorators.view_filter()
    def has_seniority(self, seniority: Literal["junior", "senior", "mid"]) -> sqlalchemy.ColumnElement:
        """Do not use argument name in function call, just the value, remember about the quotes!"""
        if seniority == "junior":
            return RECRUITMENT_MODEL.classes.candidate.years_of_experience < 2
        if seniority == "mid":
            return RECRUITMENT_MODEL.classes.candidate.years_of_experience < 5

        return RECRUITMENT_MODEL.classes.candidate.years_of_experience >= 5

    @decorators.view_filter()
    def is_from_continent(
        self, continent: Literal["Europe", "Asia", "Africa", "North America", "South America", "Australia"]
    ) -> sqlalchemy.ColumnElement:
        my_world = awoc.AWOC()
        wanted_countries = my_world.get_countries_list_of(continent)
        return RECRUITMENT_MODEL.classes.candidate.country.in_(wanted_countries)

    @decorators.view_filter()
    def data_scientist_position(self) -> sqlalchemy.ColumnElement:
        """Defines a perfect candidate for data scientist position."""
        return and_(
            RECRUITMENT_MODEL.classes.candidate.position.in_(["Data Scientist", "Machine Learning Engineer"]),
            RECRUITMENT_MODEL.classes.candidate.years_of_experience >= 3,
        )

Then, we ask a question

In [ ]:
await ask_dbally("Return all candidate suitable for data scientist position?")

## [Natural Language Response](https://db-ally.deepsense.ai/concepts/nl_responder/)

So far we could only see the executed SQL Query. For programmers, it may be enough but for the end user, the response in the natural language is much better.

Simply modify `ask_dbally` function by adding a `return_natural_reponse=True` flag to the call to the `ask` method

In [ ]:
async def ask_dbally(question: str):
    llm = LiteLLM(api_key=API_KEY)
    recruitment_db = dbally.create_collection("recruitment", llm, event_handlers=[CLIEventHandler()])
    recruitment_db.add(CandidateView, lambda: CandidateView(ENGINE))

    await recruitment_db.ask(question, return_natural_response=True)


await ask_dbally("Return all candidate suitable for data scientist position?")

Now, besides the typical conversation seen so far, you should see the prompt, retrieved table, and natural response from the agent.

```
system
You are a helpful assistant that helps answer the user's questions based on the table provided. You MUST use the table to answer the question. You are very intelligent and obedient.
The table ALWAYS contains full answer to a question.
Answer the question in a way that is easy to understand and informative.
DON'T MENTION using a table in your answer.

user
The table below represents the answer to a question: Return all candidate suitable for data scientist position?.
+------+----------------+-------------+-----------------------+---------------------------+----------------------------------------------------+---------------------------------------+--------------------------------------+
|   id | name           | country     |   years_of_experience | position                  | university                                         | skills                                | tags                                 |
|------+----------------+-------------+-----------------------+---------------------------+----------------------------------------------------+---------------------------------------+--------------------------------------|
|    1 | Emily Chen     | Canada      |                     3 | Data Scientist            | University of Toronto                              | R;Python;Machine Learning             | Data Analysis Research               |
|   10 | Daniel Kim     | South Korea |                     6 | Machine Learning Engineer | Korea Advanced Institute of Science and Technology | Python;Machine Learning;Deep Learning | Artificial Intelligence;Data Science |
|   30 | Anushka Sharma | India       |                     5 | Data Scientist            | Indian Institute of Technology Bombay              | Python;R;Machine Learning             | Data Analysis;Predictive Modeling    |
+------+----------------+-------------+-----------------------+---------------------------+----------------------------------------------------+---------------------------------------+--------------------------------------+
Answer the question: Return all candidate suitable for data scientist position?.

RESPONSE: Emily Chen from Canada and Anushka Sharma from India are suitable candidates for the Data Scientist position. Emily Chen has 3 years of experience and holds a degree from the University of
Toronto. She is proficient in R, Python, and Machine Learning, with skills in Data Analysis and Research. On the other hand, Anushka Sharma has 5 years of experience and is an alumnus of the Indian
Institute of Technology Bombay. She is skilled in Python, R, and Machine Learning, with expertise in Data Analysis and Predictive Modeling.
```
## Joined view

So far we've been querying only the Candidate table. It is quite limiting, so let's overcome it by creating a view that [joins](https://docs.sqlalchemy.org/en/14/orm/queryguide.html#joins) all three tables:

1. We need to redefine `get_select` function to return joined view:
  1. Provide only a candidate column to the select statement.
  2. join all columns on the [foreign keys](https://www.w3schools.com/sql/sql_foreignkey.asp)
2. We write filters utilizing this joined view. Notice that every filter uses a different table.

**Remark:** the only thing that's changed in the `ask_dbally` is an exchange of CandidateView to RecruitmentView

In [ ]:
class RecruitmentView(SqlAlchemyBaseView):
    """
    Joined view of the entire database, meant for answering questions mixing candidate, job_offer and application tables.
    """

    def get_select(self) -> sqlalchemy.Select:
        return (
            select(RECRUITMENT_MODEL.classes.candidate)
            .join(
                RECRUITMENT_MODEL.classes.application,
                RECRUITMENT_MODEL.classes.candidate.id == RECRUITMENT_MODEL.classes.application.candidate_id,
            )
            .join(
                RECRUITMENT_MODEL.classes.offer,
                RECRUITMENT_MODEL.classes.application.job_offer_id == RECRUITMENT_MODEL.classes.offer.id,
            )
        )

    @decorators.view_filter()
    def applied_for_company(self, company: str) -> sqlalchemy.ColumnElement:
        return RECRUITMENT_MODEL.classes.offer.company == company

    @decorators.view_filter()
    def has_application_status(self, status: str) -> sqlalchemy.ColumnElement:
        return RECRUITMENT_MODEL.classes.application.status == status

    @decorators.view_filter()
    def at_least_experience(self, years: int) -> sqlalchemy.ColumnElement:
        return RECRUITMENT_MODEL.classes.candidate.years_of_experience >= years


async def ask_dbally(question: str):
    llm = LiteLLM(api_key=API_KEY)
    recruitment_db = dbally.create_collection("recruitment", llm, event_handlers=[CLIEventHandler()])
    recruitment_db.add(RecruitmentView, lambda: RecruitmentView(ENGINE))

    await recruitment_db.ask(question, return_natural_response=True)

Now, Let's test it. Start with the example below, and try to ask another questions, see results and analyze the prompt.



In [ ]:
await ask_dbally("Which candidates with more than 3 years of experience succesfully applied to TestTech Corp?")

Notice, the obtained SQL query

```sql
SELECT candidate.id, candidate.name, candidate.country, candidate.years_of_experience, candidate.position,         
candidate.university, candidate.skills, candidate.tags                                                             
FROM candidate JOIN application ON candidate.id = application.candidate_id JOIN offer ON application.job_offer_id =
offer.id                                                                                                           
WHERE candidate.years_of_experience >= 3 AND offer.company = 'TestTech Corp' AND application.status = 'True'
```

To wrap things up, we try to ask the very first question once more

In [ ]:
await ask_dbally("Which candidates have more than 5 years of experience?")

Move back to the first response and notice the difference in the returned number of rows.

```
first response (without joined view):
  REQUEST OUTPUT:

  Number of rows: 29

second response (with joined view):
  REQUEST OUTPUT:

  Number of rows: 32

```

this time query returned more rows then previously. This is due to joined view and the fact that same candidate can apply on many offers and it counts too.

We can solve this by adding both previously defined views to the Collection [(see more info)](https://db-ally.deepsense.ai/concepts/collections/)

In [ ]:
async def ask_dbally(question: str):
    llm = LiteLLM(api_key=API_KEY)
    recruitment_db = dbally.create_collection("recruitment", llm, event_handlers=[CLIEventHandler()])
    recruitment_db.add(CandidateView, lambda: CandidateView(ENGINE))
    recruitment_db.add(RecruitmentView, lambda: RecruitmentView(ENGINE))

    await recruitment_db.ask(question, return_natural_response=True)

Shall we test it?

In [ ]:
await ask_dbally("Which candidates have more than 5 years of experience?")

Now pay attention to the additional phase that was run at the very beginning - [View Selection](https://db-ally.deepsense.ai/concepts/views/).

```
First you need to select a class to query, based on its description and the user question. You have the following  
classes to choose from:                                                                                            
CandidateView: View meant for answering questions just about the candidates.                                       
RecruitmentView: Joined view of the entire database, meant for answering questions mixing candidate, job_offer and
application tables.

...
...

RESPONSE: CandidateView
```

All defined filters with their description were given to the LLM, and it was asked to select the most appropriate view for the query. The rest is exactly the same.

## Next steps

Congratulations, we did it! We created the dbally agent that can handle the recruitment database very well.

Now it's time for you to explore more topics and extend this chatbot with them:
* [Similarity Index](https://db-ally.deepsense.ai/how-to/implementing_similarity_index/)
* [Integration with Langsmith](https://db-ally.deepsense.ai/how-to/log_runs_to_langsmith/)
* ...